#  Display options:

In [1]:

%load_ext autoreload
%autoreload 2

%xmode Minimal

#  Setting display options 
#
import pandas as pd
pd.set_option("display.width", 640)
   #
import numpy as np
np.set_printoptions(edgeitems = 30, linewidth = 100000, 
   formatter = dict(float = lambda x: "%.3g" % x))


#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate


   ###

    
#  How to use tabulate-
#
l_result = [{ "col1": 20, "col2": 30}]
   #
#  print(tabulate(l_result, headers='keys', tablefmt='psql', showindex=False))

print("--")


Exception reporting mode: Minimal


--


#  Setup stuff: Connectivity

In [2]:

from katana import remote

my_client = remote.Client()

print(my_client)


In [3]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph_BB"

print("--")


--


In [ ]:

#  Create database

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)


In [ ]:

#  Create graph
#
my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


In [4]:

#  Connect to graph

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


<_Graph my_graph_BB, CgeN7QGhx2Ao7PbLXsWD5tVC9H3KkW9jfmDTqeF88Dzq, 287>


In [ ]:

#  Now that we share servers, get a better list of databases and graphs ..
#

for l_each_d in my_client.databases():
   print("Database Name: %-28s   Id: %s" % (l_each_d.name, l_each_d.database_id))
      #
   for l_each_g in my_client.get_database(name=l_each_d.name).graphs_in_database(): 
      print("   Graph Name: %-28s      Version: %-8d   Id: %s" % (l_each_g.name, l_each_g.version, l_each_g.graph_id))


print(""  )
print("--")


#  UDFs, Part 01: Just MPI

In [ ]:

#  Best practice; Have a second NoteBook open with the Operations Widget ready-
#
#  Why ?
#     When you make mistakes below, you will want to kill the Python job that is running
#     on the worker nodes, the UDFs you submit below-
#
#  Source,
#     https://mpi4py.readthedocs.io/en/stable/tutorial.html


In [5]:

#  Example for testing; generates a random dataset
#

import time
   #
from IPython.display import clear_output


def my_func():
    
   from katana.distributed import single_host
      #
   import numpy as np                                                     #  Unparitioned objects are just fine; we are working from only
                                                                          #  one node
   l_return = np.random.randint(1, 101, 4)                                #  Generate an array[4] of random numbers in the range of 1-100

   return single_host(host=0, result=l_return)                            #  Multiple worker nodes (hosts), can only return from 1


l_cntr = 0
   #
while True:
   l_cntr += 1
   print("Running interation:  %-16d" % (l_cntr))
      #
   my_return = my_graph.run(lambda g: my_func())
   time.sleep(300) 
   clear_output(wait = True)
    


Running interation:  2               


ServerCommunicationError: request failed: http://highmem-32-katana:8080/v2/graph/CgeN7QGhx2Ao7PbLXsWD5tVC9H3KkW9jfmDTqeF88Dzq/run